In [ ]:
import numpy as np
import pandas as pd
import random
import simpy

In [5]:
# creating class1:
class Bank(object):
    def __init__(self, env, num_tellers):
        self.env = env
        self.teller = simpy.Resource(env, num_tellers)
       
    def service(self, customer):
        service_time = random.expovariate(1/10)  # Avg service time of 10 minutes
        yield self.env.timeout(service_time)

In [4]:
# function for customer data:
def customer(env, name, bank, data):
    arrival_time = env.now
    print(f'{name} arrives at the bank at {arrival_time:.2f}')
   
    with bank.teller.request() as request:
        yield request
        wait_time = env.now - arrival_time
        print(f'{name} waits for {wait_time:.2f}')
       
        yield env.process(bank.service(name))
       
        service_time = env.now - arrival_time
        print(f'{name} leaves the bank at {env.now:.2f}')
   
    # Append the customer data to the list
    data.append((name, arrival_time, wait_time, service_time))

In [3]:
# running the simulation:
def run_simulation(env, num_customers, bank, data):
    for i in range(num_customers):
        env.process(customer(env, f'Customer {i}', bank, data))
        yield env.timeout(random.expovariate(1/5))  # New customer every 5 minutes on average


In [ ]:
# Set up and run the simulation
env = simpy.Environment()
bank = Bank(env, num_tellers=3)
num_customers = 100
data = []  # This will hold the customer data
env.process(run_simulation(env, num_customers, bank, data))
env.run()

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Customer', 'ArrivalTime', 'WaitTime', 'ServiceTime'])
print(df.head())
print(df.describe())